In [1]:
import numpy as np
import lmdb
import sys
sys.path.insert(0,'../python')
import caffe
import os
import cv2
from scipy import misc
from os.path import expanduser
home_dir = expanduser("~")
coco_root = home_dir+'/data/coco'
from pycocotools.coco import COCO
from pycocotools.mask import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
annType = 'segm'
dataType='val2014'
annFile = '%s/annotations/instances_%s.json'%(coco_root,dataType)
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=10.19s)
creating index...
index created!


In [3]:
cats = cocoGt.getCatIds()
k=1
catdict={}
segm_dir = coco_root + '/preprocessed' + '/segmentation-gt'

if not os.path.exists(segm_dir):
    os.makedirs(segm_dir)
    
for catid in cats:
    catdict[catid] = k
    k = k+1
    
imgIds = cocoGt.getImgIds()
for imgId in imgIds:
    img = cocoGt.loadImgs(imgId)[0]
    Cmap = np.zeros((81,512,512))
    annIds = cocoGt.getAnnIds(imgIds=imgId)
    anns = cocoGt.loadAnns(annIds)
    for ann in anns:
        segm = ann['segmentation']
        if type(segm) == dict:
            temp = [segm]
            segm = temp
        Rs = frPyObjects( segm, img['height'], img['width'] )
        masks = decode(Rs)
        masks = masks.max(axis=2)
        catid = ann['category_id']
        Cmap[catdict[catid]] = np.round(cv2.resize(np.squeeze(masks),(512,512)))
    BGmap = np.ones((512,512)) - Cmap.max(axis=0)
    Cmap[0] = BGmap
    if not np.any(np.bitwise_or(Cmap==0,Cmap==1)):
        raise('Class map should be binary')
    outfile = segm_dir + '/' + str(imgId) 
    np.save(outfile, np.round(Cmap))
    